In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score,  f1_score, precision_score, confusion_matrix

In [ ]:
df=pd.read_csv("weatherAUS.csv")

In [ ]:
df.info() # Rzeczywiście nie ma nulli w bazie danych

## Analiza danych

Zacznijmy od podzielenia ramki danych względem taretu(*RainTomorrow*) i poszukajmy jakichś zależności między nimi.

In [ ]:
df[df.RainTomorrow==1].describe()

In [ ]:
df[df.RainTomorrow==0].describe()

Na pierwszy rzut oka najważniejszymi zmiennymi będą zachmurzenie, wilgotność oraz opady w dniu dzisiejszym, co raczej nikogo nie zdziwi. Co ciekawe ciśnienie spada, a pędkość wiatru rośnie w dni deszczowe.

In [ ]:
df.hist(bins=30,figsize=(20,10))

# Modelowanie

In [ ]:
X=df.drop(['RainTomorrow'],axis=1)
y=df['RainTomorrow']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=152)

## Logistic Regression

Zaczniemy od regresji logistycznej z prametrami:
   - penalty=l2, ponieważ chcemy aby model był zależny od jak najmniejszej liczby parametrów,
   - dual=False, ponieważ n_samples>n_features, 
   - n_jobs=-1, ponieważ jeśli coś da się zrobić szybciej to nie ma co się oganiczać,
   - class_weight=balanced, ponieważ dni bez opadów jest sporo więcej, więc chcemy trochę bardziej to zrównoważyć tą dysproporcję.

In [ ]:
model=LogisticRegression(penalty='l2',dual=False,n_jobs=-1,max_iter=1500,random_state=72,class_weight='balanced')

In [ ]:
model.fit(x_train,y_train)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
print(f"accurancy: {accuracy_score(y_test,y_pred)}")
print(f"recall score: {recall_score(y_test,y_pred)}")
print(f"f1 score: {f1_score(y_test,y_pred)}")
print(f"precision score: {precision_score(y_test,y_pred)}")
confusion_matrix(y_test,y_pred)

# RandomForestClassifier
 

Następnie sprawdzimy Las losowy z prametrami:
   - n_estimators=100,
   - min_samples_leaf=5.

In [ ]:
model3=RandomForestClassifier( n_estimators=100, random_state=10, min_samples_leaf=5)

In [ ]:
model3.fit(x_train,y_train)
y_pred3=model3.predict(x_test)

In [ ]:
print(f"accurancy: {accuracy_score(y_test,y_pred3)}")
print(f"recall score: {recall_score(y_test,y_pred3)}")
print(f"f1 score: {f1_score(y_test,y_pred3)}")
print(f"precision score: {precision_score(y_test,y_pred3)}")
confusion_matrix(y_test,y_pred3)

# VotingClassifier

Na koniec sprawdzimy VotingClassifier z prametrami:
   - estimators=[('LR',model),('RFC',model3)], ponieważ chcemy sprawdzić, czy Random Forest połączony z Regresją liniową dadzą wspólnie lepszy efekt,
   - voting='soft', 
   - n_jobs=-1, ponieważ  jeśli coś da się zrobić szybciej to nie ma co się oganiczać.

In [ ]:
model2=VotingClassifier( estimators=[('LR',model),('RFC',model3)], voting='soft',n_jobs=-1)

In [ ]:
model2.fit(x_train,y_train)
y_pred2=model2.predict(x_test)

In [ ]:
print(f"accurancy: {accuracy_score(y_test,y_pred2)}")
print(f"recall score: {recall_score(y_test,y_pred2)}")
print(f"f1 score: {f1_score(y_test,y_pred2)}")
print(f"precision score: {precision_score(y_test,y_pred2)}")
confusion_matrix(y_test,y_pred2)

Wybranie najlepszego modelu jest dość subiektywne ze względu na subiektywność, która metryka jest najlepsza dla danego zadania. Jeśli patrzymy na f1 to wygrywa VotingClassifier, najlepsze accurancy osiąga Random Forest, w recall zwycięża regresja logistyczna, a w precision score Random Forest.